# 🌸 AIみちくさチャンネル 無料配布 Flux_1_Kontext_Dev版_起動 ノートブック 🌸

# 🍀 はじめに
- ＡＩみちくさチャンネル視聴者の皆様いつも本当にありがとうございます！🐣  
- このノートブックは、最新の ComfyUI を Google Drive に設置し、Flux_1_Kontext_Dev をどなたでも簡単に起動できる様にしたものです。


## 📄 目次
1. Google Drive のマウント  
2. ComfyUI 本体の取得と更新  
3. フォルダ構成の自動生成  
4. 必要モデルのダウンロード  
5. 依存関係のインストール  
6. cloudflared の準備  
7. ComfyUI の起動  

##  1️⃣ Google Drive のマウント

In [ ]:
# 🍀 Google Drive をマウントして、モデルやファイルを Drive 上に保存します
from google.colab import drive
print("🌱 Google Drive をマウントします…")
drive.mount('/content/drive')

## 2️⃣ ComfyUI 本体の取得と更新

In [ ]:
%%bash
# 🌸 ComfyUI のワークスペースを指定
WORKSPACE="/content/drive/MyDrive/ComfyUI"

# 📂 フォルダがなければ初回クローン、あれば pull で更新
if [ ! -d "$WORKSPACE" ]; then
  echo "📂 ComfyUI を初回クローンしています…"
  git clone https://github.com/comfyanonymous/ComfyUI.git "$WORKSPACE"
else
  echo "🍀 ComfyUI を更新しています…"
  cd "$WORKSPACE" && git pull
fi

# ワークスペースに移動
cd "$WORKSPACE"

## 3️⃣ フォルダ構成の自動生成

In [ ]:
%%bash
# 🍀 Drive 上にモデルや出力用フォルダを一気に作成（存在すればスキップ）
WORKSPACE="/content/drive/MyDrive/ComfyUI"
echo "📁 フォルダ構成を準備中…"

mkdir -p \
  "$WORKSPACE/models/diffusion_models" \
  "$WORKSPACE/models/vae" \
  "$WORKSPACE/models/text_encoders" \
  "$WORKSPACE/controlnet" \
  "$WORKSPACE/embeddings" \
  "$WORKSPACE/lora" \
  "$WORKSPACE/clipVision" \
  "$WORKSPACE/output"

echo "✅ フォルダ生成完了！"

## 4️⃣ 必要モデルのダウンロード

In [ ]:
%%bash
# 🌸 各種モデルをダウンロード
WORKSPACE="/content/drive/MyDrive/ComfyUI"

echo "🌱 diffusion model をダウンロード…"
wget -c https://huggingface.co/Comfy-Org/flux1-kontext-dev_ComfyUI/resolve/main/split_files/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors \
     -P "$WORKSPACE/models/diffusion_models/"

echo "🌱 vae をダウンロード…"
wget -c https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors \
     -P "$WORKSPACE/models/vae/"

echo "🌱 text encoder をダウンロード…"
wget -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors \
     -P "$WORKSPACE/models/text_encoders/"
wget -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors \
     -P "$WORKSPACE/models/text_encoders/"


##5️⃣ 依存関係のインストール

In [ ]:
%%bash
# 📃 ComfyUI ディレクトリに移動して依存関係をインストール
cd "/content/drive/MyDrive/ComfyUI"
echo "📃 依存関係をインストール中…"
pip install -q xformers!=0.0.18 -r requirements.txt \
  --extra-index-url https://download.pytorch.org/whl/cu121 \
  --extra-index-url https://download.pytorch.org/whl/cu118 \
  --extra-index-url https://download.pytorch.org/whl/cu117
echo "✅ 依存関係インストール完了！"


##6️⃣ cloudflared の準備

In [ ]:
%%bash
# 🌎 cloudflared をインストール
echo "🌎 cloudflared をインストールしています…"
# Cloudflare の公開鍵を追加
curl -fsSL https://pkg.cloudflare.com/pubkey.gpg | sudo apt-key add -
# apt リポジトリを追加
echo 'deb https://pkg.cloudflare.com/ jammy main' | sudo tee /etc/apt/sources.list.d/cloudflare.list
# パッケージリストを更新＆cloudflaredをインストール
sudo apt-get update -q
sudo apt-get install -y cloudflared

echo "✅ cloudflared のインストールが完了しました！"

##7️⃣ ComfyUI の起動_（cloudflared の準備_トンネル起動）

In [ ]:
import os
import subprocess
import threading
import time
import socket
from google.colab import drive, output

# 1️⃣ Google Drive をマウント
print("🌱 Google Drive をマウントしています…")
drive.mount('/content/drive', force_remount=True)

# 2️⃣ ComfyUI リポジトリのクローン or 更新
WORKSPACE = '/content/drive/MyDrive/ComfyUI'
if not os.path.isdir(WORKSPACE):
    print("📂 ComfyUI をクローンしています…")
    subprocess.run(['git', 'clone', '--depth', '1', 'https://github.com/comfyanonymous/ComfyUI.git', WORKSPACE])
else:
    print("🔄 ComfyUI を最新に更新しています…")
    subprocess.run(['git', 'pull'], cwd=WORKSPACE)

# 3️⃣ 依存関係のインストール
print("📦 依存関係をインストールしています…（少し時間がかかります）")
subprocess.run([
    'pip', 'install', '-q', 'xformers!=0.0.18', '-r', 'requirements.txt',
    '--extra-index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://download.pytorch.org/whl/cu118',
    '--extra-index-url', 'https://download.pytorch.org/whl/cu117'
], cwd=WORKSPACE)

# 4️⃣ ComfyUI をバックグラウンドで起動
print("🔧 ComfyUI を起動しています…")
subprocess.Popen(
    ['python', 'main.py', '--listen', '--host', '0.0.0.0', '--port', '8188'],
    cwd=WORKSPACE
)

# 5️⃣ ポート 8188 がオープンするのを待って iframe 埋め込み
def serve_comfy():
    print("⏳ ポート 8188 の準備を待っています…")
    while True:
        time.sleep(0.5)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            if sock.connect_ex(('127.0.0.1', 8188)) == 0:
                break
    print("✅ ComfyUI が起動しました！UI を埋め込みます…")
    output.serve_kernel_port_as_iframe(8188, height=900)
    output.serve_kernel_port_as_window(8188)

threading.Thread(target=serve_comfy, daemon=True).start()
